In [23]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf

import numpy

zip_file = tf.keras.utils.get_file(origin="https://www.kaggle.com/moltean/fruits?resource=download&downloadHash=14b8c2fc75851029ee0ee978058a11d1270b0bd13642ad3b32dad593617612eb", fname="archive.zip", extract=True)
base_dir, _ = os.path.splitext('/Users/eamspoker21/fruits-360')


train_dir = os.path.join(base_dir, 'Training')
validation_dir = os.path.join(base_dir, 'Test')

image_size = 128
batch_size = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(directory=train_dir, target_size=(image_size, image_size), batch_size=batch_size)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(directory=validation_dir, target_size=(image_size, image_size), batch_size=batch_size)

IMG_SHAPE = (image_size, image_size, 3)

base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, include_top=False)
base_model.trainable = False

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(131, activation='softmax')
])

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 1
steps_per_epoch = numpy.ceil(train_generator.n / batch_size)
validation_steps = numpy.ceil(validation_generator.n / batch_size)

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Model)   (None, 4, 4, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 131)               134275    
Total params: 3,363,139
Trainable params: 134,275
Non-trainable params: 3,228,864
_________________________________________________________________
2116/2116 [==============================] - 4676s 2s/step - loss: 0.1492 - accuracy: 0.9746 - val_loss: 9.0001 - val_accuracy: 0.0196


In [26]:
saved_model_dir = 'content/TFLite'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

INFO:tensorflow:Assets written to: content/TFLite/assets
